In [1]:
import numpy as np
import copy
# import triangular solvers from previous workbook
%run 'WS03 - Triangular systems and Gaussian elimination.ipynb'

[[1.]
 [2.]
 [2.]]
[[1.]
 [2.]
 [2.]]
x0= [1. 1. 1. 1.]
x= [1. 1. 1. 1.]
[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 -8.8817842e-16]


In [33]:
'''*** Jacobi ***'''
def Jacobi_iteration(A, b, max_iteration, x0 = None):
    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')

    # check diagonal is non zero
    for i in range(n):
        if np.isclose(A[i, i], 0):
            raise ValueError(f'A[{i}, {i}] is zero')

    # construct iteration matrices
    P=np.zeros([n,n])    # matrix P = D^{-1}(L+U)
    p=np.zeros(n)        # vector p = D^{-1} b
    for i in range(n):
        p[i]=b[i]/A[i,i] 
        for j in range(n):
             P[i,j] = A[i,j]/A[i,i]
        P[i,i] = 0
        
    #create a new array to store the results, initialised as zero
    if x0 is None:
        x = np.zeros_like(b)
    else:
        x = x0.copy()
    
    # perform iteration x <- p - P * x
    for it in range(max_iteration):
        xnew = np.empty_like(x)
        for i in range(n):
            xnew[i] = p[i]
            for j in range(n):
                xnew[i] -= P[i, j] * x[j]
        x = xnew.copy()
                
    return x

In [34]:
'''*** Gauss-Seidel ***'''
Gauss_Seidel_iteration(A, b, max_iteration, x0 = None):
    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')

    for i in range(n):
        if np.isclose(A[i, i], 0):
            raise ValueError(f'A[{i}, {i}] is zero')

    # do not construct iteration matrices explicitly
    LD = np.zeros_like(A)
    U = np.zeros_like(A)
    for i in range(n):
        for j in range(n):
            if i < j:
                U[i, j] = A[i, j]
            else:
                LD[i, j] = A[i, j]
    
    # p = (L + D)^{-1} b --> found by solving triangular system
    # (L + D) p = b
    p = lower_triangular_solve(LD, b)
      
    #create a new array to store the results, initialised as zero
    if x0 is None:
        x = np.zeros_like(b)
    else:
        x = x0.copy()
        
    # perform iteration x <- p - P * x
    # (L+D)(xnew - p) = U*x
    Ux = np.empty_like(x)
    for it in range(max_iteration):
        for i in range(n):
            Ux[i] = 0.0
            for j in range(i+1, n):
                Ux[i] += U[i, j] * x[j]
        Px = lower_triangular_solve(LD, Ux)
        x = p - Px
                
    return x

In [35]:
'''Testing'''
# Test different linear solvers starting from the above two-dimensional linear system
A = np.array([[2, 1], [1, 2]])
b = np.array([3, 3])
x_exact = np.array([1,1])

# numpy linear solver
x0 = np.linalg.solve(A,b)
print("Solution by numpy solver:", x0)

x = Jacobi_iteration(A, b, 4)
print("Solution by Jacobi iteration: ",x)
print("Error: ", x - x_exact)
print("Residual: ", np.matmul(A,x)-b)

x = Gauss_Seidel_iteration(A, b, 4)
print("Solution by Gauss Seidel iteration: ",x)
print("Error: ", x - x_exact)
print("Residual: ", np.matmul(A,x)-b)

# Note we do not expect these values to be very small for small numbers of iterations!

Solution by numpy solver: [1. 1.]
Solution by Jacobi iteration:  [0.9375 0.9375]
Error:  [-0.0625 -0.0625]
Residual:  [-0.1875 -0.1875]
Solution by Gauss Seidel iteration:  [1.0078125  0.99609375]
Error:  [ 0.0078125  -0.00390625]
Residual:  [0.01171875 0.        ]


In [36]:
'''Testing 2'''
A = np.array([[-10, 2, 0, 67], [-2, 50, -77, 1.e-5], [1, 7, 30, 8], [-10, -7, 0.001, 80]])
b = np.array([1, 2, 9, 0])

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = Jacobi_iteration(A, b, 100)
print("Solution by Jacobi iteration: ",x)
print("Residual: ", np.matmul(A,x)-b)
# larger tolerance needed since the convergence is slower!
np.testing.assert_allclose(x, x0, rtol=1.0e-4)

x = Gauss_Seidel_iteration(A, b, 100)
print("Solution by Gauss Seidel iteration: ",x)
print("Residual: ", np.matmul(A,x)-b)
np.testing.assert_allclose(x, x0)

Solution by numpy solver: [0.9244595  0.31826746 0.15668124 0.14340388]
Solution by Jacobi iteration:  [0.92445478 0.31826793 0.15668179 0.1434019 ]
Residual:  [-8.49516542e-05 -9.70662051e-06 -8.42150747e-07 -1.14962922e-04]
Solution by Gauss Seidel iteration:  [0.9244595  0.31826746 0.15668124 0.14340388]
Residual:  [ 6.37090380e-11  3.58131302e-11  7.74846853e-12 -1.77635684e-15]


In [37]:
'''Testing 3'''
n=20
B = np.random.rand(n, n)
eps = 10
A = eps * np.eye(n) + B * B.T
b = np.random.rand(n)

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = Jacobi_iteration(A, b, 100)
print("Solution by Jacobi iteration: ",x)
print("Residual: ", np.matmul(A,x)-b)
np.testing.assert_allclose(x, x0)

x = Gauss_Seidel_iteration(A, b, 100)
print("Solution by Gauss Seidel iteration: ",x)
print("Residual: ", np.matmul(A,x)-b)
np.testing.assert_allclose(x, x0)

Solution by numpy solver: [ 0.01342526  0.08307362  0.01856768  0.03086629  0.00642853 -0.00630217
  0.07935638  0.03028664  0.04399549  0.01314382  0.04842941  0.05635187
  0.06244147 -0.00977329  0.06737911  0.04851531  0.02962853  0.03029331
  0.01765032  0.01390566]
Solution by Jacobi iteration:  [ 0.01342526  0.08307362  0.01856768  0.03086629  0.00642853 -0.00630217
  0.07935638  0.03028664  0.04399549  0.01314382  0.04842941  0.05635187
  0.06244147 -0.00977329  0.06737911  0.04851531  0.02962853  0.03029331
  0.01765032  0.01390566]
Residual:  [-5.55111512e-17  3.33066907e-16  0.00000000e+00  0.00000000e+00
 -5.55111512e-17  6.93889390e-18 -4.44089210e-16  1.11022302e-16
  1.11022302e-16  0.00000000e+00  1.11022302e-16  1.11022302e-16
  0.00000000e+00 -2.77555756e-17  3.33066907e-16  2.22044605e-16
 -1.11022302e-16  0.00000000e+00 -5.55111512e-17  0.00000000e+00]
Solution by Gauss Seidel iteration:  [ 0.01342526  0.08307362  0.01856768  0.03086629  0.00642853 -0.00630217
  0.07